In [4]:
"""
This cell will load in the data, run the algorithm once, and print out the Sharpe Ratio
"""

import zipline
from zipline.api import order_target_percent
import pytz
from datetime import datetime
import matplotlib.pyplot as pyplot
from collections import defaultdict

from zipline import TradingAlgorithm
from zipline.api import order_target, record, symbol, history
import numpy as np
import pandas as pd

#: NOTICE HOW THIS IS OUTSIDE INITIALIZE, BECAUSE IT IS, WE CAN REDEFINE IT EVERYTIME WE REDINE INITIALIZE
short_mavg_days = 20
long_mavg_days = 50

def initialize(context):
    context.aapl = 24
    context.spy = 8554
    
    # Set context parameters based on global parameters 
    context.short_mavg_days = short_mavg_days
    context.long_mavg_days = long_mavg_days
    
    # Register 2 histories that track daily prices,
    # one with a short window and one with a long window
    add_history(short_mavg_days, '1d', 'price')
    add_history(long_mavg_days, '1d', 'price')

    # Used to warm-up moving averages
    context.i = 0
    
def handle_data(context, data):
    
    # Skip days to get full windows
    context.i += 1
    if context.i < context.long_mavg_days:
        return
    
    # Compute averages
    # history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = history(context.short_mavg_days, '1d', 'price').mean()
    long_mavg = history(context.long_mavg_days, '1d', 'price').mean()

    #sym = symbol('AAPL')

    # Trading logic
    if short_mavg[context.aapl] > long_mavg[context.aapl]:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target_percent(context.aapl, 1)
    elif short_mavg[context.aapl] < long_mavg[context.aapl]:
        order_target_percent(context.aapl, 0)


data = get_pricing(
    ['AAPL','SPY'],
    start_date='2014-01-01',
    end_date = '2015-02-15',
    frequency='daily'
)

algo_obj = TradingAlgorithm(
    initialize=initialize, 
    handle_data=handle_data
)

#: Run the backtest and save the result.
perf_manual = algo_obj.run(data.transpose(2,1,0))

#: Get the sharpe ratio
sharpe = (perf_manual.returns.mean()*252)/(perf_manual.returns.std() * np.sqrt(252))
print ("The Sharpe ratio is {}").format(sharpe)

NameError: name 'get_pricing' is not defined

In [ ]:
# This will create a list with 9 entries
short_mavg_days = [days for days in np.arange(5, 50, 5)]
# This will create a list with 40 entries
long_mavg_days = [days for days in np.arange(10, 210, 5)]

#: Create a dictionary to hold all the results of our algorithm run
all_sharpes = defaultdict(dict)

# Count the number of backtests run
backtest_count = 0

# This will loop and run 324 backtests
# Each backtest takes about 3 seconds, so this will take around 16 minutes
for short_mavg_day in short_mavg_days:
    for long_mavg_day in long_mavg_days:
        # Only consider cases where the short is less than long.. but why this not working?
        if short_mavg_day < long_mavg_day:
            
            #: Redefine initialize with new weights
            def initialize(context):
                context.aapl = 24
    
                # Set context parameters based on global parameters 
                context.short_mavg_days = short_mavg_day
                context.long_mavg_days = long_mavg_day
    
                # Register 2 histories that track daily prices,
                # one with a short window and one with a long window
                add_history(context.short_mavg_days, '1d', 'price')
                add_history(context.long_mavg_days, '1d', 'price')

                # Used to warm-up moving averages
                context.i = 0  
      
            algo_obj = TradingAlgorithm(
                initialize=initialize, 
                handle_data=handle_data
            )
            perf_manual = algo_obj.run(data.transpose(2,1,0))
            
            # Keep track of how many backtests were run
            backtest_count += 1
            print("Backtest {0} completed...").format(backtest_count)
            
            # Calculate the sharpe for this backtest
            sharpe = (perf_manual.returns.mean()*252)/(perf_manual.returns.std() * np.sqrt(252))
        
            #: Add the result to our dict
            all_sharpes[short_mavg_day][long_mavg_day] = sharpe


print " "
print "All backtest simulations completed!"
print " "

all_sharpes = pd.DataFrame(all_sharpes)
all_sharpes.index.name = "Long Moving Average Days"
all_sharpes.columns.name = "Short Moving Average Days"

In [ ]:
import matplotlib.pyplot as pyplot

def heat_map(df):
    """
    This creates our heatmap using our sharpe ratio dataframe
    """
    fig = pyplot.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values,cmap = pyplot.get_cmap('RdYlGn'), interpolation = 'nearest')
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index))
    ax.set_title("Sharpe Ratios")
    pyplot.colorbar(axim)
    
#: Plot our heatmap
heat_map(all_sharpes)

print all_sharpes

In [ ]:
Max_Sharpe = max(all_sharpes.max(axis=1))
print("Maximum Sharpe = {0:.2f}").format(Max_Sharpe)

In [ ]:
all_sharpes